In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import classification_report, confusion_matrix
import logging
import torch
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [14]:
def contact_detection_accuracy(df, allowable_delay = 20):
    "df should include label, majority voting and time"
    TP, TN, FP, FN = 0, 0, 0, 0
    
    df['label_diff'] = df['label'].diff().fillna(0)
    df['TP'], df['TN'], df['FP'], df['FN'] = None, None, None, None
 
    contact_delays = []
    no_contact_delays = []
    df = df.reset_index(drop=True)
 
    change_events = df[df['label_diff'] != 0].index.tolist()
 
    # Add first and last index if not present
    if 0 not in change_events:
        change_events.insert(0, 0)
    if len(df) - 1 not in change_events:
        change_events.append(len(df) - 1)
 
    for event_idx in range(len(change_events) - 1):
        idx = change_events[event_idx]
        
        if df.label[idx] == 1: # Starting a contact event
            for i in range(idx, min(idx + allowable_delay, len(df))):
                if df.majority_voting[i] == 1:
                    delay = df.time[i] - df.time[idx]
                    contact_delays.append(delay)
                    break
            
            if i >= (idx + allowable_delay - 1):
                contact_delays.append(0)
 
            # Calculate TP and FN
            for j in range(change_events[event_idx+1], max(change_events[event_idx+1]-allowable_delay, 0), -1):
                if df.majority_voting[j] == 1:
                    break
 
            for idx_ in range(i, j):
                if df.majority_voting[idx_] == df.label[idx_]:
                    TP += 1
                    df.loc[idx_, 'TP'] = 0.9
                else:
                    FN += 1
                    df.loc[idx_, 'FN'] = 0.9
 
        else: # Starting a contact-free event
 
            for i in range(idx, min(idx + allowable_delay, len(df))):
                if df.majority_voting[i] == 0:
                    delay = df.time[i] - df.time[idx]
                    no_contact_delays.append(delay)
                    break
            
            if i >= (idx + allowable_delay - 1):
                no_contact_delays.append(0)
 
            # Calculate TN and FP
            for j in range(change_events[event_idx+1], max(change_events[event_idx+1]-allowable_delay, 0), -1):
                if df.majority_voting[j] == 0:
                    break
                
            for idx_ in range(i, j):
                if df.majority_voting[idx_] == df.label[idx_]:
                    TN += 1
                    df.loc[idx_, 'TN'] = 0.1
                else:
                    FP += 1
                    df.loc[idx_, 'FP'] = 0.1
 
    # Compute averages safely
    contact_avg_delay = np.nanmean([d for d in contact_delays if d > 0])
    no_contact_avg_delay = np.nanmean([d for d in no_contact_delays if d > 0])
 
    ModelAccuracy = (TP + TN) / (TP + TN + FP + FN) * 100
    DetectionFailureRate = FN / (TP + FN) * 100
    FalseAlarmRate = FP / (TN + FP) * 100
 
    return df, TP, TN, FP, FN, contact_avg_delay, no_contact_avg_delay, ModelAccuracy, DetectionFailureRate, FalseAlarmRate, contact_delays, no_contact_delays
 

In [17]:
# 单个文件夹处理
df = pd.read_csv('../DATA/Labeled_data/0305_STFT3DCNN_Post123_T4ESBN4+7_P/labeled_modelResult_data.csv')
# Call the function
df, TP, TN, FP, FN, contact_avg_delay, no_contact_avg_delay, ModelAccuracy, DetectionFailureRate, FalseAlarmRate, contact_delays, no_contact_delays = contact_detection_accuracy(df, 20)
 
print('contact_delays:', contact_delays)
print('no_contact_delays:', no_contact_delays)
print(f"Accuracy:{ModelAccuracy:.2f}%, DetectionFailureRate:{DetectionFailureRate:.2f}%, FalseAlarmRate:{FalseAlarmRate:.2f}%, Contact Detection Delay: {contact_avg_delay*1000:.3f}ms, No-Contact Detection Delay: {no_contact_avg_delay*1000:.3f}ms, TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
 
# Plot results
df['Model Output(Contact Detection)'] = df['majority_voting']



df[['time', 'label', 'Model Output(Contact Detection)', 'TP', 'TN', 'FP', 'FN']].iplot(
    x='time', kind='scatter',
    mode={'label': 'lines', 'Model Output(Contact Detection)': 'lines', 'TP': 'markers', 'TN': 'markers', 'FP': 'markers', 'FN': 'markers'},
    title='Contact Detection Analysis (Interactive)',
    xTitle='Time Index', yTitle='Signal', theme='white',
    symbol=[None, None, 'x', 'x', 'x', 'x'],
    size = 5,
    colors=['blue', 'pink', 'green', 'lightgreen', 'orange', 'red'],
    layout=dict(
        width=800,          # 图的总宽度，可根据期刊单栏宽度调整
        height=500,         # 图的总高度，保证有足够空间给上半部分放图例

        # 让绘图区只占据图像下半部分 (这里设置成 0 ~ 0.5)
        xaxis=dict(domain=[0, 1], title='Time Index'),
        yaxis=dict(domain=[0, 0.95], title='Signal'),

        # 将图例放在上半部分，水平排列
        legend=dict(
            orientation='h',   # 水平放置
            x=0.5,             # 水平居中
            y=1.0,             # 紧贴画布顶部
            xanchor='center',
            yanchor='bottom'
        ),

        # 边距，避免标题或图例贴到图边缘
        margin=dict(
            l=60,  # 左边距
            r=20,  # 右边距
            t=100, # 上边距，根据实际需求调大或调小
            b=60   # 下边距
        ),

        # 字体大小
        font=dict(size=20)
    )
)

df

contact_delays: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
no_contact_delays: [0.0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0]
Accuracy:52.81%, DetectionFailureRate:67.69%, FalseAlarmRate:38.89%, Contact Detection Delay: nanms, No-Contact Detection Delay: nanms, TP: 486, TN: 2269, FP: 1444, FN: 1018


/tmp/ipykernel_1254036/3782058487.py:71: RuntimeWarning:

Mean of empty slice

/tmp/ipykernel_1254036/3782058487.py:72: RuntimeWarning:

Mean of empty slice



,index,time,tau_J0,tau_J1,tau_J2,tau_J3,tau_J4,tau_J5,tau_J6,tau_J_d0,...,label,model_result_time,touch_type_idx,majority_voting,label_diff,TP,TN,FP,FN,Model Output(Contact Detection)
0,1,0.000000,-0.037866,-36.200062,0.536611,16.276621,0.327171,2.289623,-0.028515,0.0,...,0,0.000000,0.0,0,0.0,None,0.1,None,None,0
1,2,0.004997,-0.002160,-36.124683,0.536611,16.236948,0.336326,2.279552,-0.028515,0.0,...,0,0.007003,0.0,0,0.0,None,0.1,None,None,0
2,3,0.009996,-0.002160,-36.124683,0.536611,16.236948,0.336326,2.279552,-0.028515,0.0,...,0,0.012384,0.0,0,0.0,None,0.1,None,None,0
3,4,0.015004,-0.002160,-36.164356,0.608022,16.312326,0.327171,2.289623,-0.018444,0.0,...,0,0.012384,0.0,0,0.0,None,0.1,None,None,0
4,5,0.020004,-0.002160,-36.164356,0.608022,16.312326,0.327171,2.289623,-0.018444,0.0,...,0,0.017746,0.0,0,0.0,None,0.1,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5593,5594,27.964994,-0.264001,-36.200062,0.496938,16.312326,0.318015,2.289623,-0.099926,0.0,...,0,27.966748,0.0,0,0.0,None,0.1,None,None,0
5594,5595,27.970004,-0.264001,-36.200062,0.496938,16.312326,0.318015,2.289623,-0.099926,0.0,...,0,27.972808,0.0,0,0.0,None,0.1,None,None,0
5595,5596,27.975000,-0.264001,-36.200062,0.496938,16.387705,0.308860,2.310680,-0.038585,0.0,...,0,27.972808,0.0,0,0.0,None,0.1,None,None,0
5596,5597,27.979991,-0.264001,-36.200062,0.496938,16.387705,0.308860,2.310680,-0.038585,0.0,...,0,27.979036,0.0,0,0.0,None,0.1,None,None,0
